In [1]:
import numpy as np

In [2]:
grofile = 'Complex_I-cg.gro'

with open(grofile) as gro:
    gro.readline()
    natoms = int(gro.readline())
    atoms = [ gro.readline() for _ in range(natoms) ]

bb = np.loadtxt([ line[20:] for line in atoms if 'BB' in line ])
bb2id = np.array([ idx for idx, line in enumerate(atoms) if 'BB' in line ])

In [3]:
breaks = np.where((np.diff(bb, axis=0) ** 2).sum(axis=1) > 0.20)[0] + 1
chains = np.split(bb, breaks)

In [4]:
distances = {
    (i, j): ((pi[:, None] - pj[None, :])**2).sum(axis=2)
    for i, pi in enumerate(chains[:-1])
    for j, pj in enumerate(chains[i+1:], i+1)
}
contacts = { pair: np.where(dists < 1) for pair, dists in distances.items() }

In [5]:
chainshift = [0, *np.cumsum([len(c) for c in chains])]
itp = open('elastic_band.itp', "w")
#print(';[ intermolecular_interactions ]')
for i in range(len(chains)):
    for j in range(i+1, len(chains)):
        print(f'; Elastic bonds for proteins {i} and {j}', file=itp)
        print(*[
            f'{ai + 1 :6d}\t{aj + 1 :6d}\t1\t{d:8.5f}\t500' for ai, aj, d in zip(
                bb2id[contacts[(i, j)][0] + chainshift[i]],
                bb2id[contacts[(i, j)][1] + chainshift[j]],
                distances[(i,j)][contacts[(i,j)][0], contacts[(i, j)][1]] ** 0.5
        )
        ], sep='\n', file=itp)


In [6]:
#contacts[(0, 1)]
bb2id[contacts[(0, 1)][0]]

array([], dtype=int64)